# Strategy with Calibration and Flexible Crypto List
- Large (5)
BTC, ETH, SOL, XRP, AVAX
Why: mega-caps leading or participating in the 2025 alt rally (BTC dominance high; ETH/XRP outperformance waves; SOL/AVAX remain top L1 growth bets). 
CoinGecko
+1
CoinDesk

- Mid (5)
NEAR, INJ, RNDR, TIA, ONDO
Why: strong “builder” narratives and periodic surges—NEAR/INJ execution, RNDR (AI rendering), TIA (modular/L1 data-availability), ONDO (RWA/treasury-like flows). 
CoinGecko

- Emerging (5)
WIF, PEPE, JUP, PENDLE, SEI
Why: high beta + liquidity; JUP (Solana infra), PENDLE (yield-tokenization DeFi), SEI (trading-focused L1), plus momentum memecoins (WIF, PEPE) that can drive outsized short-term moves.

In [6]:
# "2025-09-07T00:00:00Z" in utc
start_datetime = "2025-08-05T12:00:00Z"
end_datetime = "2025-09-08T22:00:00Z"
metadata_path = (
    "trading_bot_development/bot_modules_calib/config/ticker_metadata_crypto.json"
)
default_path = "default_params_821.json"
ticker_path = f"ticker_params.json"

save_data = False

## Imports

In [ ]:
import os
import importlib
import sys
import pandas as pd
import json

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.helper_functions import (
    clear_pycache, 
    setup_autoreload, 
    delete_symbol_csvs, 
    unnest_conf_indicators, 
    safe_ratio_calc,
    save_df_with_metadata
)

clear_pycache()
setup_autoreload()


%load_ext autoreload
%autoreload 2
import utils.eda_utils as eda

from strategy_definition_crypto.crypto_bot.strategy_engine import (
    StrategyEngine,
)
from strategy_definition_crypto.crypto_bot.data_fetcher import (
    DataFetcher,
)
from strategy_definition_crypto.crypto_bot.signal_generator import (
    SignalGenerator,
)
from crypto_bot.strategy_params import (
    StrategyParams,
)
from crypto_bot.trade_logger import (
    TradeLogger,
)

from crypto_bot.parameter_manager import (
    ParameterManager,
)

class DummyRiskManager:
    def assess_trade(self, direction, latest_price):
        print(
            f"[Risk] Assessing trade direction={direction}, price={latest_price['close']}"
        )
        return {"approve": True, "side": "buy", "symbol": "AAPL", "quantity": 10}


class DummyBroker:
    def place_order(self, side, symbol, quantity):
        print(f"[Broker] Placing {side.upper()} order for {symbol}, qty={quantity}")

API_KEY = "SGUFIZB1UFKXLVDE"
risk_manager = DummyRiskManager()
broker = DummyBroker()
trade_logger = TradeLogger("trades_backtest.db")

strategy_metadata_path = (
    "trading_bot_development/tests_metadata.json"
)
with open(strategy_metadata_path, "r") as f:
    strategy_metadata = json.load(f)
test_date = strategy_metadata.get("test_date", "unknown_test_date")
signal_discovery_strategy = strategy_metadata.get("signal_discovery_strategy", "unknown_signal_discovery_strategy")
default_parameters = strategy_metadata.get("default_parameters", "unknown_default_parameters")
timeframes = strategy_metadata.get("timeframes", "unknown_timeframes")
confirmation_strategy = strategy_metadata.get("confirmation_strategy", "unknown_confirmation_strategy")

🧹 Removed 0 __pycache__ directories.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 Autoreload is enabled.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
with open(metadata_path, "r") as f:
    metadata = json.load(f)
symbols_all = [ticker for ticker, _ in metadata.items()]

start_date = start_datetime.split(" ")[0]
end_date = end_datetime.split(" ")[0]
ticker_set_name = metadata_path.split("/")[-1]

all_trades_results = None
failed = []
empty = []

for symbol in symbols_all:
    company_type = metadata[symbol]["group"]
    for term_trade in ["short", "medium"]:  # , "long"]:
        files_paths = {
            "1min": f"data_all/{symbol}_1min_{start_date}.csv",
        }
        try:
            print(f"\nProcessing {symbol} for {term_trade} trade")
            params = ParameterManager.load_params(
                symbol=symbol,
                term_trade=term_trade,
                company_type=company_type,
                ticker_path=ticker_path,
                default_path=default_path,
            )
            signal_generator_obj = SignalGenerator(params)
            data_fetcher_obj = DataFetcher(
                symbol, API_KEY, files_paths, start_datetime, end_datetime
            )
            engine_obj = StrategyEngine(
                data_fetcher_obj,
                signal_generator_obj,
                risk_manager,
                broker,
                params,
                # trade_logger,
            )
            trigger_price_multiplier = 1.001
            trades_df = engine_obj.backtest_strategy_modular(
                start_datetime, trigger_price_multiplier=trigger_price_multiplier
            )

            if not trades_df.empty and not trades_df.isna().all().all():
                trades_df["symbol"] = symbol
                trades_df["company_type"] = company_type
                if all_trades_results is None:
                    all_trades_results = trades_df.copy()
                else:
                    all_trades_results = pd.concat(
                        [all_trades_results, trades_df], ignore_index=True
                    )
            else:
                empty.append((symbol, term_trade))

        except Exception as e:
            failed.append((symbol, term_trade, str(e)))
            print(f"Failed to process {symbol} for {term_trade} trade: {e}")

    delete_symbol_csvs(files_paths)

if trade_logger:
    trade_logger.close()


Processing BTC for short trade
fetch_all_timeframes

Processing BTC for medium trade
fetch_all_timeframes

Processing ETH for short trade
fetch_all_timeframes

Processing ETH for medium trade
fetch_all_timeframes

Processing SOL for short trade
fetch_all_timeframes

Processing SOL for medium trade
fetch_all_timeframes

Processing XRP for short trade
fetch_all_timeframes

Processing XRP for medium trade
fetch_all_timeframes

Processing AVAX for short trade
fetch_all_timeframes

Processing AVAX for medium trade
fetch_all_timeframes

Processing NEAR for short trade
fetch_all_timeframes

Processing NEAR for medium trade
fetch_all_timeframes

Processing INJ for short trade
fetch_all_timeframes

Processing INJ for medium trade
fetch_all_timeframes

Processing RNDR for short trade
fetch_all_timeframes
Failed to process RNDR for short trade: single positional indexer is out-of-bounds

Processing RNDR for medium trade
fetch_all_timeframes
Failed to process RNDR for medium trade: single positio

In [9]:
all_trades_results

,signal_detection_time,entry_time,exit_time,direction,term_trade_type,confirmation,signal_detected_price,entry_price,exit_price,mod_count,fast_df_count,confirmation_df_count,conf_indicators,prev_low,now_low_current_price,next_low,symbol,company_type
0,2025-08-06 13:42:00,2025-08-06 13:47:00,2025-08-06 13:51:00,bullish,short,confirmed,114133.2800,114247.413280,114050.345352,0,"[1, 1, 1, 1]","[3, 3, 3, 3]","{'adx': (-0.014217198941585048, 18.4), 'di': (...",114057.1700,113933.1600,113903.6600,BTC,large_cap_tech
1,2025-08-06 14:57:00,2025-08-06 14:57:00,2025-08-06 15:04:00,bullish,short,confirmed,114829.4800,114944.309480,115005.430686,2,"[1, 1, 1, 1]","[3, 3, 3, 3]","{'adx': (0.12812303316368034, 14.7), 'di': ((0...",115085.9700,114977.4500,114913.8900,BTC,large_cap_tech
2,2025-08-07 14:39:00,NaT,NaT,bullish,short,not_triggered,116600.0400,NaN,NaN,0,"[1, 1, 1, 1]","[3, 3, 3, 3]","{'adx': (-0.0605067122756764, 16.9), 'di': ((0...",NaN,NaN,NaN,BTC,large_cap_tech
3,2025-08-08 15:27:00,NaT,NaT,bullish,short,not_triggered,116538.9400,NaN,NaN,0,"[1, 1, 1, 1]","[3, 3, 3, 3]","{'adx': (-0.05092631931078596, 24.8), 'di': ((...",NaN,NaN,NaN,BTC,large_cap_tech
4,2025-08-10 09:51:00,2025-08-10 12:17:00,2025-08-10 12:52:00,bullish,short,confirmed,118432.7600,118551.192760,118640.784652,11,"[1, 1, 1, 1]","[3, 3, 3, 3]","{'adx': (0.03389850373124752, 17.7), 'di': ((0...",118681.8500,118612.9000,118521.9700,BTC,large_cap_tech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4818,2025-09-07 15:30:00,2025-09-07 15:31:00,2025-09-07 15:40:00,bullish,medium,confirmed,0.2982,0.298498,0.297371,0,"[3, 3, 3, 3]","[5, 5, 5, 5]","{'adx': (0.0781564161210754, 22.2), 'di': ((0....",0.2977,0.2973,0.2977,SEI,emerging_volatile
4819,2025-09-07 15:35:00,2025-09-07 15:35:00,2025-09-07 15:39:00,bullish,medium,confirmed,0.2985,0.298798,0.297700,0,"[3, 3, 3, 3]","[5, 5, 5, 5]","{'adx': (0.07533317023791479, 23.9), 'di': ((-...",0.2978,0.2977,0.2973,SEI,emerging_volatile
4820,2025-09-08 12:45:00,2025-09-08 12:50:00,2025-09-08 13:21:00,bullish,medium,confirmed,0.2996,0.299900,0.300337,6,"[3, 3, 3, 3]","[5, 5, 5, 5]","{'adx': (0.13851207600934382, 27.7), 'di': ((0...",0.3007,0.3003,0.3006,SEI,emerging_volatile
4821,2025-09-08 12:50:00,2025-09-08 12:50:00,2025-09-08 13:21:00,bullish,medium,confirmed,0.2998,0.300100,0.300337,6,"[3, 3, 3, 3]","[5, 5, 5, 5]","{'adx': (0.10726868391461071, 30.9), 'di': ((-...",0.3007,0.3003,0.3006,SEI,emerging_volatile


## Portfolio Performance Summary

In [ ]:
if failed:
    print(f"\nFailed trades: {len(failed)}")
    for f in failed:
        print(f)

importlib.reload(eda)
initial_portfolio = 10000
play_size = 3000
if not all_trades_results.empty:
    all_trades_results = all_trades_results.sort_values(by="entry_time").reset_index(
        drop=True
    )
    bullish_trades_df = all_trades_results.copy()
    bullish_trades_df = bullish_trades_df[bullish_trades_df["direction"] == "bullish"]
    bullish_trades_df = bullish_trades_df.reset_index(drop=True)

    bullish_trades_df.drop(columns=["direction"], inplace=True)
    bullish_trades_df["shares"] = play_size / bullish_trades_df["entry_price"]
    bullish_trades_df["pnl_usd"] = (
        bullish_trades_df["exit_price"] - bullish_trades_df["entry_price"]
    ) * bullish_trades_df["shares"] - play_size * 0.001 * 2
    # bullish_trades_df = bullish_trades_df[bullish_trades_df["pnl_usd"] > 60]
    eda.analyze_strategy_performance_multiple_tickers_updated(
        bullish_trades_df, play_size, initial_portfolio
    )
    if len(bullish_trades_df[bullish_trades_df["confirmation"] == "confirmed"]) > 0:
        bullish_trades_df["trade_duration_min"] = (
            bullish_trades_df["exit_time"] - bullish_trades_df["entry_time"]
        ).dt.total_seconds() / 60
        bullish_trades_df["pnl_pct_per_trade"] = round(
            (bullish_trades_df["pnl_usd"] / play_size * 100), 3
        )
        bullish_trades_df.sort_values(by="pnl_usd", inplace=True)
        confirmed_bullish_trades_df = bullish_trades_df[
            bullish_trades_df["confirmation"] == "confirmed"
        ].copy()
        confirmed_bullish_trades_df.sort_values(
            by="signal_detection_time", inplace=True
        )
        display(confirmed_bullish_trades_df)

        conf_df = confirmed_bullish_trades_df[
            [
                "conf_indicators",
                "signal_detection_time",
                "symbol",
                "pnl_usd",
                "company_type",
                "term_trade_type",
                "trade_duration_min",
            ]
        ].copy()
        unnested_df = unnest_conf_indicators(conf_df)
        unnested_df["ratio"] = unnested_df.apply(safe_ratio_calc, axis=1)
        if save_data:
            try:
                save_df_with_metadata(
                    unnested_df,
                    f"sep_conf_indicators_summary_{test_date}.csv",
                )
            except ValueError as e:
                print(f"Column mismatch error: {e}")
    else:
        unnested_df = pd.DataFrame()
    metadata_test = {
        "start_date": start_date,
        "end_date": end_date,
        "strategy_version": signal_discovery_strategy,
        "default_parameters": default_parameters,
        "timeframes": timeframes,
        "confirmation_strategy": confirmation_strategy,
        "ticker_set_name": ticker_set_name,
    }
    performance_summary = eda.get_strategy_performance_summary(
        bullish_trades_df, play_size, initial_portfolio
    )
    if save_data:
        try:
            save_df_with_metadata(
                performance_summary,
                f"sep_performance_summary_detailed_{test_date}.csv",
                metadata_dict=metadata_test,
                append=True,
            )
        except ValueError as e:
            print(f"Column mismatch error: {e}")


Failed trades: 2
('RNDR', 'short', 'single positional indexer is out-of-bounds')
('RNDR', 'medium', 'single positional indexer is out-of-bounds')

==== Overall Performance (Capital-Constrained) ====
Initial Portfolio Value:       $10000.00
Cash per Play:                 $3000
Max Cash in Play at Once:      $18000 (6 concurrent trades)

Final Portfolio Value:         $8727.92
Total PnL:                     $-1272.08 (-12.72%)

Trades Actually Executed:      1093
Trades Skipped (No Capital):   3384
Total Confirmed Signals:       4477
Total Signals:                 4823
Not Triggered Signals:         346
Win rate:                      31.20%

Average PnL per Trade:          $-1.16 (-0.04%)
Average Portfolio Impact:       -0.01%


== Best Trade ==
Ticker: PENDLE
PnL:  $141.28 (4.71%)
Time: 2025-08-14 15:15:00 → 2025-08-14 15:55:00

== Worst Trade ==
Ticker: INJ
PnL: $-56.26 (-1.88%)
Time: 2025-08-14 12:48:00 → 2025-08-14 12:53:00


==== Performance by Term Type ====


,trade_count,avg_pnl_usd,avg_pnl_pct,avg_portfolio_impact,total_pnl_usd,win_rate
term_trade_type,,,,,,
medium,230,0.43,0.01,0.00,98.79,34.35
short,863,-1.59,-0.05,-0.02,-1370.87,30.36




==== Trade Duration Stats ====
Average Trade Duration: 0 days 00:14:46.770356816

Shortest Trade:   0 days 00:01:00
Longest Trade:    0 days 01:34:00

Effective Trading Days: 34 of 34
Average Trades per Day: 32.1


==== Capital Constraint Impact ====
Missed PnL due to capital constraints: $-2590.19
Potential total PnL (if unlimited capital): $-3862.27
Capital efficiency: 24.4%

Top symbols missed due to capital constraints:
  SEI: $453.47
  ETH: $332.89
  PENDLE: $187.23
  AVAX: $64.66
  INJ: $-52.99


,signal_detection_time,entry_time,exit_time,term_trade_type,confirmation,signal_detected_price,entry_price,exit_price,mod_count,fast_df_count,...,conf_indicators,prev_low,now_low_current_price,next_low,symbol,company_type,shares,pnl_usd,trade_duration_min,pnl_pct_per_trade
44,2025-08-06 12:03:00,2025-08-06 13:31:00,2025-08-06 13:33:00,short,confirmed,2.95190,2.954852,2.949016,0,"[1, 1, 1, 1]",...,"{'adx': (0.025797633037417247, 13.9), 'di': ((...",2.94960,2.94480,2.94630,XRP,large_cap_tech,1.015279e+03,-7.924840,2.0,-0.264
1,2025-08-06 12:03:00,2025-08-06 12:04:00,2025-08-06 12:10:00,short,confirmed,0.00001,0.000010,0.000010,1,"[1, 1, 1, 1]",...,"{'adx': (0.01574981839290872, 20.0), 'di': ((0...",0.00001,0.00001,0.00001,PEPE,emerging_volatile,2.909712e+08,-9.905001,6.0,-0.330
85,2025-08-06 12:03:00,2025-08-06 14:05:00,2025-08-06 14:13:00,short,confirmed,2.46700,2.469467,2.474251,1,"[1, 1, 1, 1]",...,"{'adx': (-0.01624427054681282, 12.2), 'di': ((...",2.47800,2.47400,2.47100,NEAR,moderate_midcap,1.214837e+03,3.811281,8.0,0.127
89,2025-08-06 12:03:00,2025-08-06 14:06:00,2025-08-06 14:14:00,short,confirmed,12.92000,12.932920,12.935356,2,"[1, 1, 1, 1]",...,"{'adx': (-0.0483451236426774, 9.8), 'di': ((0....",12.94000,12.93000,12.93000,INJ,moderate_midcap,2.319662e+02,-1.434854,8.0,-0.048
83,2025-08-06 12:03:00,2025-08-06 14:05:00,2025-08-06 14:13:00,short,confirmed,21.99000,22.011990,22.052895,1,"[1, 1, 1, 1]",...,"{'adx': (-0.042513001506156806, 12.0), 'di': (...",22.06000,22.03000,22.00000,AVAX,large_cap_tech,1.362894e+02,3.574899,8.0,0.119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4471,2025-09-08 15:21:00,2025-09-08 15:21:00,2025-09-08 15:32:00,short,confirmed,215.59000,215.805590,216.068752,3,"[1, 1, 1, 1]",...,"{'adx': (0.04235341521781255, 23.2), 'di': ((0...",216.13000,215.98000,216.09000,SOL,large_cap_tech,1.390140e+01,1.658327,11.0,0.055
4473,2025-09-08 15:24:00,2025-09-08 15:25:00,2025-09-08 15:37:00,short,confirmed,1.72600,1.727726,1.722577,3,"[1, 1, 1, 1]",...,"{'adx': (-0.007801960285352204, 40.6), 'di': (...",1.72400,1.72100,1.72000,TIA,moderate_midcap,1.736386e+03,-10.940530,12.0,-0.365
4474,2025-09-08 15:24:00,2025-09-08 15:25:00,2025-09-08 15:34:00,short,confirmed,0.52020,0.520720,0.520300,1,"[1, 1, 1, 1]",...,"{'adx': (0.040755339486165776, 28.9), 'di': ((...",0.52040,0.51890,0.51900,JUP,emerging_volatile,5.761251e+03,-4.420878,9.0,-0.147
4476,2025-09-08 15:27:00,2025-09-08 15:28:00,2025-09-08 15:34:00,short,confirmed,13.60000,13.613600,13.562447,0,"[1, 1, 1, 1]",...,"{'adx': (-0.004588820792834534, 31.1), 'di': (...",13.57000,13.55000,13.55000,INJ,moderate_midcap,2.203679e+02,-13.272408,6.0,-0.442


In [20]:
positive_pnl = confirmed_bullish_trades_df[confirmed_bullish_trades_df["pnl_usd"] >= 20]
negative_pnl = confirmed_bullish_trades_df[confirmed_bullish_trades_df["pnl_usd"] < 10]
positive_sum = positive_pnl["pnl_usd"].sum()
positive_count = positive_pnl["pnl_usd"].count()
negative_sum = negative_pnl["pnl_usd"].sum()
negative_count = negative_pnl["pnl_usd"].count()
print(f"Positive trades: {positive_count}, Total PnL: {positive_sum}")
print(f"Negative trades: {negative_count}, Total PnL: {negative_sum}")

Positive trades: 430, Total PnL: 19143.50862960117
Negative trades: 3736, Total PnL: -27430.386879863716


# Confirmation Indicators

In [ ]:
# unnested_df.to_csv(f"conf_indicators_summary.csv", index=False)

## Results by group


In [12]:
print("==== Performance by Symbol ====")
grouped_symbol = (
    bullish_trades_df[bullish_trades_df["confirmation"] == "confirmed"]
    .groupby("company_type")
    .agg(
        trade_count=("pnl_usd", "count"),
        avg_pnl_usd=("pnl_usd", "mean"),
        # avg_pnl_pct=("pnl_pct", "mean"),
        total_pnl_usd=("pnl_usd", "sum"),
        win_rate=("pnl_usd", lambda x: (x > 0).sum() / len(x) * 100),
    )
)
grouped_symbol.sort_values(by="total_pnl_usd", ascending=False, inplace=True)
grouped_symbol

==== Performance by Symbol ====


,trade_count,avg_pnl_usd,total_pnl_usd,win_rate
company_type,,,,
large_cap_tech,1156,-0.266991,-308.642036,33.304498
moderate_midcap,1374,-1.098284,-1509.042874,33.042213
emerging_volatile,1947,-1.050121,-2044.584894,33.281972


In [13]:
print("==== Performance by Company and Term ====")
grouped = (
    bullish_trades_df[bullish_trades_df["confirmation"] == "confirmed"]
    .groupby(["symbol", "term_trade_type"])
    .agg(
        trade_count=("pnl_usd", "count"),
        avg_pnl_usd=("pnl_usd", "mean"),
        # avg_pnl_pct=("pnl_pct", "mean"),
        total_pnl_usd=("pnl_usd", "sum"),
        win_rate=("pnl_usd", lambda x: (x > 0).sum() / len(x) * 100),
    )
)
grouped.sort_values(by="total_pnl_usd", ascending=False, inplace=True)
grouped.reset_index(inplace=True)
grouped

==== Performance by Company and Term ====


,symbol,term_trade_type,trade_count,avg_pnl_usd,total_pnl_usd,win_rate
0,PENDLE,short,287,2.267068,650.648466,39.372822
1,ETH,medium,53,6.223257,329.832641,47.169811
2,AVAX,medium,90,3.209203,288.828270,37.777778
3,PENDLE,medium,120,1.813783,217.653977,39.166667
4,SEI,medium,103,1.794555,184.839173,33.980583
5,SEI,short,270,0.562791,151.953507,37.037037
6,INJ,medium,80,1.772055,141.764361,41.250000
7,BTC,medium,19,2.584632,49.108009,42.105263
8,AVAX,short,226,0.156137,35.286921,34.955752
9,SOL,medium,84,0.353085,29.659114,36.904762


### Adding company type and parameters' type

In [14]:
ticker_full_path = f"trading_bot_development/bot_modules_calib/config/{ticker_path}"

with open(ticker_full_path, "r") as f:
    ticker_params = json.load(f)

ticker_df = pd.DataFrame(
    index=metadata.keys(),
    columns=["long", "short", "medium"],
    data=0,
).astype(int)

for ticker, params in ticker_params.items():
    for term in ["long", "short", "medium"]:
        if term in params:
            ticker_df.at[ticker, term] = 1
if not ticker_df.empty:
    grouped["calibrated_parameters"] = grouped.apply(
        lambda row: ticker_df.loc[row["symbol"], row["term_trade_type"]], axis=1
    )
    confirmed_bullish_trades_df["calibrated_parameters"] = (
        confirmed_bullish_trades_df.apply(
            lambda row: ticker_df.loc[row["symbol"], row["term_trade_type"]], axis=1
        )
    )
else:
    grouped["calibrated_parameters"] = "default"
    confirmed_bullish_trades_df["calibrated_parameters"] = "default"

grouped["company_type"] = grouped["symbol"].map(
    lambda x: metadata[x]["group"] if x in metadata else "unknown"
)
confirmed_bullish_trades_df["company_type"] = confirmed_bullish_trades_df["symbol"].map(
    lambda x: metadata[x]["group"] if x in metadata else "unknown"
)
grouped

,symbol,term_trade_type,trade_count,avg_pnl_usd,total_pnl_usd,win_rate,calibrated_parameters,company_type
0,PENDLE,short,287,2.267068,650.648466,39.372822,0,emerging_volatile
1,ETH,medium,53,6.223257,329.832641,47.169811,0,large_cap_tech
2,AVAX,medium,90,3.209203,288.828270,37.777778,0,large_cap_tech
3,PENDLE,medium,120,1.813783,217.653977,39.166667,0,emerging_volatile
4,SEI,medium,103,1.794555,184.839173,33.980583,0,emerging_volatile
5,SEI,short,270,0.562791,151.953507,37.037037,0,emerging_volatile
6,INJ,medium,80,1.772055,141.764361,41.250000,0,moderate_midcap
7,BTC,medium,19,2.584632,49.108009,42.105263,0,large_cap_tech
8,AVAX,short,226,0.156137,35.286921,34.955752,0,large_cap_tech
9,SOL,medium,84,0.353085,29.659114,36.904762,0,large_cap_tech


In [55]:
# confirmed_bullish_trades_df.to_csv("trades_27morn.csv", index=False)

In [15]:
grouped_calibrated = (
    grouped.groupby(["calibrated_parameters"])
    .agg(
        total_pnl_usd=("total_pnl_usd", "sum"),
    )
    .reset_index()
)
grouped_calibrated.sort_values(by="total_pnl_usd", ascending=False, inplace=True)
grouped_calibrated

,calibrated_parameters,total_pnl_usd
0,0,-3862.269804


In [16]:
grouped_company_type = grouped.groupby("company_type").agg(
    trade_count=("trade_count", "sum"),
    avg_pnl_usd=("total_pnl_usd", "mean"),
    total_pnl_usd=("total_pnl_usd", "sum"),
    win_rate=("win_rate", "mean"),
)
grouped_company_type.sort_values(by="total_pnl_usd", ascending=False, inplace=True)
grouped_company_type.reset_index(inplace=True)
grouped_company_type

,company_type,trade_count,avg_pnl_usd,total_pnl_usd,win_rate
0,large_cap_tech,1156,-30.864204,-308.642036,35.050750
1,moderate_midcap,1374,-188.630359,-1509.042874,34.286954
2,emerging_volatile,1947,-204.458489,-2044.584894,33.601841


In [17]:
grouped_company_term_type = grouped.groupby(["company_type", "term_trade_type"]).agg(
    trade_count=("trade_count", "sum"),
    avg_pnl_usd=("total_pnl_usd", "mean"),
    total_pnl_usd=("total_pnl_usd", "sum"),
    win_rate=("win_rate", "mean"),
)
grouped_company_term_type.sort_values(by="total_pnl_usd", ascending=False, inplace=True)
grouped_company_term_type.reset_index(inplace=True)
grouped_company_term_type

,company_type,term_trade_type,trade_count,avg_pnl_usd,total_pnl_usd,win_rate
0,large_cap_tech,medium,292,107.041544,535.207719,37.574132
1,moderate_midcap,medium,343,20.393960,81.575841,36.869091
2,emerging_volatile,medium,515,-126.810504,-634.052520,34.463604
3,large_cap_tech,short,864,-168.769951,-843.849755,32.527368
4,emerging_volatile,short,1432,-282.106475,-1410.532373,32.740078
5,moderate_midcap,short,1031,-397.654679,-1590.618715,31.704817
